# Workbook : Machine Learning

For our last section workbook (so that next week you can ask questions about and work on your final projects in section), we're going to work with a dataset all about craft beer. We'll work to predict what type of beer each is based on the characteristics of that beer.

**Disclaimer**: Working with data about beer does *NOT* mean that I'm encouraging the drinking of beer by students. In fact, your professor doesn't even like beer (blech). Specifically, individuals under the age of 21 are not legally allowed to consume alcoholic beverages, but lucky for you all, that doesn't stop us from working with data on the topic!

The data we'll use here come from a publicly-available [Kaggle dataset on craft beer](https://www.kaggle.com/nickhould/craft-cans).

# Part I : Data, Wrangling, & EDA

To get started, you'll need to **import the following**:
   * `pandas` as `pd`
   * `numpy` as `np`
   * from `sklearn.svm`: `SVC` 
   * from `sklearn.metrics`: `confusion_matrix`, `classification_report`, `precision_recall_fscore_support` 

In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

In [3]:
assert pd
assert np
assert SVC
assert confusion_matrix
assert classification_report
assert precision_recall_fscore_support

Now that you're setup to go in Python, **read in the `'breweries.csv'` file from the `data/` directory. Assign this to the variable `breweries`**. Then, **read in the file `beers.dsv` from the `data/` directory. Assign this to the variable `beers`.**

In [24]:
beers = pd.read_csv("data/beers.csv")
breweries = pd.read_csv("data/breweries.csv")

In [25]:
assert breweries.shape == (558, 4)
assert beers.shape == (2410, 8)

Run the code below to take a **look at the first few rows of each dataset** to give yourself an idea of what data are inclued in each dataset. Notice if there are any common columns between the two datasets.

In [26]:
breweries.head()

,Unnamed: 0,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [27]:
beers.head()

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


To get a quick handle on what's going on these data, **print out the number of missing values in each variable of the variables in the `beers` dataset .**

In [28]:
beers.isnull().sum()

Unnamed: 0       0
abv             62
ibu           1005
id               0
name             0
style            5
brewery_id       0
ounces           0
dtype: int64

We're going to try to predict the `style` of beer from its alcohol by volume (`abv`) and its international bitterness unites (`ibu`). To do this, **remove any beers from our `beers` dataset where data are missing for any of these three values. Store this back into hte `beers` dataset.** 

Note that you may not always want to take this approach and removing samples from your dataset will not always be appropriate, but for this example, it's a reasonable approach.

In [29]:
beers = beers.dropna(how='any', subset=['style', 'abv','ibu'])

In [30]:
assert beers.shape == (1403, 8)

Using the `beers` dataset you've not got, **merge `beers` and `breweries` together using a left join. Assign this to the variable `beer_df`. Be sure to look at the first few rows of `beer_df`.**

In [34]:
beer_df = pd.merge(beers, breweries, how='left')

beer_df.head()

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces,city,state
0,14,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),177,12.0,NaN,NaN
1,21,0.099,92.0,1036,Lower De Boom,American Barleywine,368,8.4,NaN,NaN
2,22,0.079,45.0,1024,Fireside Chat,Winter Warmer,368,12.0,NaN,NaN
3,24,0.044,42.0,876,Bitter American,American Pale Ale (APA),368,12.0,NaN,NaN
4,25,0.049,17.0,802,Hell or High Watermelon Wheat (2009),Fruit / Vegetable Beer,368,12.0,NaN,NaN


In [35]:
assert beer_df.shape == (1403, 10)

**Use and take a look at the output of the `describe()` method to describe the quantitative variables in your `beer_df` dataset.**

In [37]:
beer_df.describe()

,Unnamed: 0,abv,ibu,id,brewery_id,ounces
count,1403.000000,1403.000000,1403.000000,1403.000000,1403.000000,1403.000000
mean,1241.128297,0.059919,42.739843,1413.888810,223.375624,13.510264
std,691.675612,0.013585,25.962692,757.572191,150.387510,2.254112
min,14.000000,0.027000,4.000000,1.000000,0.000000,8.400000
25%,681.500000,0.050000,21.000000,771.000000,95.500000,12.000000
50%,1228.000000,0.057000,35.000000,1435.000000,198.000000,12.000000
75%,1864.500000,0.068000,64.000000,2068.500000,350.000000,16.000000
max,2408.000000,0.125000,138.000000,2692.000000,546.000000,32.000000


**Be sure to look at the output you just generated. What do you learn? Do any values surprise you? Are there any with really big standard deviations? Does this make sense?** (Feel free to edit this cell with any observations/notes)

Now, let's take a look and **see how many different styles of beer we have in our datset.** The `value_counts` method may help you accomplish this.

In [41]:
beer_df["style"].value_counts()

American IPA                      301
American Pale Ale (APA)           153
American Amber / Red Ale           77
American Double / Imperial IPA     75
American Blonde Ale                61
                                 ... 
Smoked Beer                         1
Roggenbier                          1
Other                               1
Old Ale                             1
English Stout                       1
Name: style, Length: 90, dtype: int64

Due to limitations in time here in section, let's just try to predict the four most common `style`s of beer. **Filter your `beer_df` dataset to only include entries from the four most common `style`s of beer.** Store this filtered dataset into `beer_df`.

In [47]:
def standardize_style(string):
     
    string = str(string)
    if 'American IPA' in string:
        output = 'American IPA'
    elif 'American Pale Ale (APA)' in string:
        output = 'American Pale Ale (APA)'
    elif 'American Amber / Red Ale' in string:
        output = 'American Amber / Red Ale'
    elif 'American Double / Imperial IPA' in string:
        output = 'American Double / Imperial IPA'
        
    else:
        output = np.nan
        
    
    return output

beer_df['style'] = beer_df['style'].apply(standardize_style)
beer_df = beer_df.dropna(how='any', subset=['style'])

In [57]:
assert beer_df.shape == (606, 10)

# Part II : Prediction Model

Let's start to build our model! To do so, **create a variable `num_training` that includes the number of samples that corresponds to 80% of our total samples in our `beer_df` dataset. Be sure that this is an integer. Also, create a variable `num_testing` including the number corresponding to 20% of our total samples.**

In [63]:
num_training = int(len(beer_df)*0.8)
num_testing = len(beer_df)- num_training

In [64]:
assert num_training == 484
assert num_testing == 122

To model these data, **split your data into `beer_X`, which includes the `abv` and `ibu` columns from `beer` (predictors). This should be a `pandas` DataFrame. The outcome variable will be `style`. Assign the outcome variable to the variable `beer_Y`. This should be a `numpy` array.**

In [66]:
beer_X = beer_df.loc[:,['abv','ibu']]
beer_Y = np.array(beer_df.loc[:,'style'])

In [67]:
assert type(beer_Y) == np.ndarray
assert beer_Y.shape == (606,)
assert beer_X.shape == (606, 2)

Before running our model, we'll need to **split our data into a training and test set. Use `num_training` (created above) to extract the following variables**: 
* from `beer_X`, generate : `beer_train_X`, `beer_test_X`
* from `beer_Y`, generate: `beer_train_Y`, `beer_test_Y`

In [69]:
beer_train_X = beer_X.iloc[:num_training]
beer_test_X = beer_X.iloc[num_training:]
beer_train_Y = beer_Y[:num_training]
beer_test_Y = beer_Y[num_training:]

In [ ]:
assert len(beer_train_X) == 484
assert len(beer_test_X) == 122

To train our model, we'll use a linear SVM classifier. Here a function has been defined for you. **Run the following cell, but be sure you understand what the function is doing.**

In [70]:
def train_SVM(X, y, kernel='linear'):
    clf = SVC(kernel=kernel)
    clf.fit(X, y)
    
    return clf

Using the `train_SVM` function defined above, **train your model. Assign this output to `beer_clf`.**

In [72]:
beer_clf = train_SVM(beer_train_X,beer_train_Y)

In [73]:
assert isinstance(beer_clf, SVC)
assert hasattr(beer_clf, "predict")

Now, **generate predictions from your training and test sets of predictors using the `predict` method. Assign your predictions from the training data to `beer_predicted_train_Y`. Assign your predictison from the test data to `beer_predicted_test_Y`.**

In [77]:
beer_predicted_train_Y = beer_clf.predict(beer_train_X)
beer_predicted_test_Y = beer_clf.predict(beer_test_X)

In [81]:
assert beer_predicted_train_Y.shape == (484,)
assert beer_predicted_test_Y.shape == (122,)

# Part III : Model Assessment

At this point, you should have built your model and generated predictions using that model for both your training and test datasets. 

Let's determine how our predictor did. **Generate a `classification_report` for the predictions generated for your training data relative to the truth (from the original beers dataset). Print the output.**

In [84]:
report_dict = classification_report(beer_train_Y,beer_predicted_train_Y, output_dict=True)
pd.DataFrame(report_dict)

,American Amber / Red Ale,American Double / Imperial IPA,American IPA,American Pale Ale (APA),accuracy,macro avg,weighted avg
precision,0.815789,0.764706,0.692308,0.566434,0.667355,0.709809,0.685071
recall,0.449275,0.245283,0.838983,0.642857,0.667355,0.544100,0.667355
f1-score,0.579439,0.371429,0.758621,0.602230,0.667355,0.577930,0.649964
support,69.000000,53.000000,236.000000,126.000000,0.667355,484.000000,484.000000


What are precision and recall? What do these numbers represent? How accurate are our predictions?

**Generate a `classification_report` for the predictions generated for your *test* data relative to the truth (from the original beers dataset). Print the output.**

In [85]:
report_dict2 = classification_report(beer_test_Y,beer_predicted_test_Y, output_dict=True)
pd.DataFrame(report_dict2)

,American Amber / Red Ale,American Double / Imperial IPA,American IPA,American Pale Ale (APA),accuracy,macro avg,weighted avg
precision,0.625,0.777778,0.701493,0.552632,0.655738,0.664225,0.677288
recall,0.625,0.318182,0.723077,0.777778,0.655738,0.611009,0.655738
f1-score,0.625,0.451613,0.712121,0.646154,0.655738,0.608722,0.644832
support,8.000,22.000000,65.000000,27.000000,0.655738,122.000000,122.000000


How is our model performing? Does tHis dffer between training and test data? Where does it have trouble? Where does it perform well? Do we have thoughts as to why? One way to determine where a model is going wrong is to look at a confusion matrix. **Generate a confusion matrix for the training data predictions as well as the ground truth from the `beer_df` dataset.**

In [88]:
confusion_matrix(beer_train_Y,beer_predicted_train_Y)

array([[ 31,   1,  10,  27],
       [  0,  13,  40,   0],
       [  0,   3, 198,  35],
       [  7,   0,  38,  81]])

**Generate a confusion matrix for the testing data.**

In [89]:
confusion_matrix(beer_test_Y,beer_predicted_test_Y)

array([[ 5,  0,  2,  1],
       [ 1,  7, 14,  0],
       [ 0,  2, 47, 16],
       [ 2,  0,  4, 21]])

While this is a somewhat small example using a limited dataset for prediction, we hope you have a better understanding of how to approach a machine learning question, knowing specifically what training and test datasets are used for, how to build a model, and how to assess model/prediction performance. **Feel free to try different models, include more beer types in your analysis or ask a completely different prediction question!**